In [1]:
# Importation des bilbiothèques
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Chargement des données
# nb de solutions : regrouper ?

url = './data_G/data_G.solution'
df = pd.read_csv(url, sep='\s+', header=None)
df.head()

#Teste si les colonnes contiennent des valeurs nulles
#df.columns[df.isnull().any()]
#df.shape

#def clean_data(data):
    #"""
    #Regarde si des nan sont présentes dans des colonnes, si oui, les supprimer.
    #Détecter s'il s'agit d'une classification ou d'une régression et appliquer le bon algorithme, 
    #en choisissant les bons paramètres.
    #Supprimer les variables peu informatives ou fortement corélées (TD maths, corrélation accumulatives...).
    #Appliquer éventuellement des techniques de réduction dimensionnelle (PCA, t-SNE) pour visualiser et décider
    #"""


,0
0,1
1,0
2,0
3,0
4,0


In [2]:
import pandas as pd
# Lecture des features (.data) dans un DataFrame pandas
data_df = pd.read_csv('./data_A/data_A.data', sep='\s+', header=None, na_values='NaN')
print("Dimensions des données du dossier A :", data_df.shape)
print(data_df.head())
# Lecture des cibles (.solution)
solution_df = pd.read_csv('./data_A/data_A.solution', sep='\s+', header=None)
print("Dimensions des solutions :", solution_df.shape)
print(solution_df.head())
# Lecture des types (.type)
with open('./data_A/data_A.type', 'r') as f:
    feature_types = [line.strip() for line in f.readlines()]
print("Nombre de features annoncées :", len(feature_types))
print("Exemples de types de features :", feature_types[:5])

Dimensions des données du dossier A : (34190, 24)
     0   1   2    3   4         5    6    7   8   9   ...  14  15    16  17  \
0  11.0  45  34  1.0  55  127921.0  1.0  1.0   0   4  ...   9   1     0   1   
1   4.0  50  41  1.0  60  231619.0  1.0  2.0   0   2  ...  10   5     0   1   
2   2.0  40  36  1.0  26  119941.0  1.0  1.0   0   4  ...   9   7     0   1   
3   1.0  40  26  1.0  38  215766.0  1.0  1.0   0   2  ...  10   2  3103   1   
4   1.0  40  28  1.0  38  170525.0  1.0  1.0   0   7  ...  11   2     0   2   

   18        19  20   21  22  23  
0   3  241885.0   0  1.0  44   4  
1   3  104334.0   0  1.0  59   9  
2   2   77953.0   0  1.0  44  15  
3   3  167350.0   0  1.0  31   4  
4   2  109857.0   0  1.0  18   1  

[5 rows x 24 columns]
Dimensions des solutions : (34190, 3)
   0  1  2
0  1  1  1
1  1  1  0
2  0  1  1
3  1  1  0
4  1  0  1
Nombre de features annoncées : 24
Exemples de types de features : ['Categorical', 'Numerical', 'Numerical', 'Categorical', 'Numerical']


In [3]:
# Détection des valeurs manquantes par colonne
missing_counts = data_df.isna().sum()
print("Valeurs manquantes par colonne :")
print(missing_counts[missing_counts > 0])
# Exemple: séparation entraînement/validation (20% validation)

from sklearn.model_selection import train_test_split
X = data_df
# features
y = solution_df # cibles (DataFrame ou Series)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print("Taille du train set :", X_train.shape, "| Taille du set de validation :", X_val.shape)

Valeurs manquantes par colonne :
0     1959
3      614
6     1952
7     1941
13     590
21    1979
dtype: int64
Taille du train set : (27352, 24) | Taille du set de validation : (6838, 24)


In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
# Séparation des colonnes numériques et catégorielles d’après feature_types
numeric_features = [i for i, t in enumerate(feature_types) if t == 'Numerical']
categorical_features = [i for i, t in enumerate(feature_types) if t == 'Categorical']
# Transformer pour les numériques: impute (moyenne) puis standardisation -> remplace NaN par la moyenne de la colonne
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('scaler', StandardScaler())])

# Transformer pour les catégorielles : impute (remplace NaN par la catégorie la plus fréquente) puis one-hot encode et ignore si catégorie inconnue en test
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

# Combinaison des transformations par colonnes
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),('cat', categorical_transformer, categorical_features)])
X_train_prepared = preprocessor.fit_transform(X_train)
X_val_prepared = preprocessor.transform(X_val)
print("Dimensions des features après prétraitement :", X_train_prepared.shape)

Dimensions des features après prétraitement : (27352, 215)


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,HistGradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
# Préparation des données pour entraînement (on suppose X_train_prepared déjà calculé via preprocessor)
X_train_arr = X_train_prepared # peut être une array numpy ou un sparse matrix selon onehot
X_val_arr = X_val_prepared
# Détection du cas multi-label vs multi-class vs binaire
y_train_arr = y_train.values # convert DataFrame to numpy array
y_val_arr = y_val.values
multilabel = False
if y_train_arr.ndim > 1 and y_train_arr.shape[1] > 1:
    multilabel = True
# Définition des modèles à tester
models = {}
# Pour logistic regression, si multi-label, on encapsule dans OneVsRest (OVR)
if multilabel:
    models['LogisticRegression_OVR'] = OneVsRestClassifier(LogisticRegression(max_iter=1000))
else:
    models['LogisticRegression'] = LogisticRegression(max_iter=1000)
    models['RandomForest'] = RandomForestClassifier(n_estimators=200)
    models['HistGradientBoosting'] = HistGradientBoostingClassifier()
# Entraînement et évaluation sur le set de validation
best_model_name = None
best_score = -1
for name, clf in models.items():
    clf.fit(X_train_arr, y_train_arr)
    # Prédiction sur validation
    y_pred = clf.predict(X_val_arr)
    # Si multi-label, accuracy_score n’est pas directement adaptée (on pourrait calculer par étiquette ou utiliser jaccard)
    # Ici on calcule l'accuracy globale moyenne pour simplifier
    if multilabel:
        score = (y_pred == y_val_arr).mean()
        best_model = clf
        print("Score : ", score)
    else:
        score = accuracy_score(y_val_arr, y_pred)
        print(f"{name} accuracy = {score:.4f}")
        if score > best_score:
            best_score = score
            best_model_name = name
            best_model = clf
            print("Meilleur modèle sélectionné :", best_model_name, "avec score validation =", best_score)

Score :  0.853855903285561


In [10]:
from sklearn.metrics import classification_report, confusion_matrix
# Supposons best_model et best_model_name obtenus précédemment
y_pred_val = best_score.predict(X_val_arr)
if multilabel:
    # Pour chaque étiquette, calculer F1-score
    print("Rapport de classification multi-label (par étiquette) :")
    for i in range(y_val_arr.shape[1]):
        print(f"Étiquette {i}:")
        print(classification_report(y_val_arr[:, i], y_pred_val[:, i]))
else:
    print("Rapport de classification :")
    print(classification_report(y_val_arr, y_pred_val))
    print("Matrice de confusion :")
    print(confusion_matrix(y_val_arr, y_pred_val))

AttributeError: 'int' object has no attribute 'predict'